# Stable Baselines3 - Car Racing

In [1]:
# !apt-get update -qq && apt-get install swig cmake -qq
!pip install moviepy
!apt-get install swig
!pip install box2d-py -q
!pip install gym
!pip install -q "stable-baselines3[extra]"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


## Importações

In [2]:
import gymnasium as gym
import numpy as np
import pandas as pd
import numpy as np

from stable_baselines3.common.vec_env import DummyVecEnv

### Importando Algoritmo e Política

In [3]:
from stable_baselines3 import PPO

#### Função Auxiliar

In [4]:
def evaluate(model, num_episodes=10, deterministic=True, qi=0):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    vec_env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = vec_env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            # also note that the step only returns a 4-tuple, as the env that is returned
            # by model.get_env() is an sb3 vecenv that wraps the >v0.26 API

            #obs, reward, done, info = vec_env.step(action)
            obs, reward, done, info = vec_env.step(action)

            episode_rewards.append(reward)

            #done = terminated or truncated


        all_episode_rewards.append(sum(episode_rewards))
        df_treinos.loc[qi][i + 1] = sum(episode_rewards)[0]

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

### Método de avaliação do Stable Baselines

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

### Criando Ambiente

In [6]:
# render_mode="human" Se usar visualização na avaliação
env = gym.make("CarRacing-v2", continuous=False)
env = DummyVecEnv([lambda: env])

In [7]:
# Espaço de observação
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [8]:
# Espaço de ações
env.action_space

Discrete(5)

### Avaliando agente não treinado

In [9]:
# Separate env for evaluation
eval_env = gym.make("CarRacing-v2", continuous=False)
model = PPO("CnnPolicy", env)

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,
)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/home/melot/.local/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=-93.04 +/- 0.4451632081869383


## Treina o agente

### Inicialização

In [10]:
# Define quantidade de treinamentos do agente
quantidades = [2e4, 5e4, 1e5, 2e5, 4e5]
qtd_teste = 10

In [11]:
df_treinos = pd.DataFrame(index=quantidades, columns=np.arange(1, qtd_teste + 1))

### Execução

In [17]:
for qtd in quantidades:
    print(qtd)
    # Algoritmo + Politica
    model = PPO("CnnPolicy", env)

    # Treina agente
    model.learn(total_timesteps=qtd)

    # Salva modelo
    model.save(f"./{qtd}k_car_racing_PPO")

    # Carrega modelo
    #model = PPO.load(f"./{qtd}k_car_racing_PPO", env)

    # Avalia modelo
    evaluate(model, num_episodes=qtd_teste, qi=qtd)


    # Deleta modelo
    del model

    # Reseta Ambiente
    env.reset()
    print()



20000.0
Mean reward: -24.9728 Num episodes: 10

50000.0
Mean reward: 462.34772 Num episodes: 10

100000.0
Mean reward: -30.031702 Num episodes: 10

200000.0
Mean reward: 866.6138 Num episodes: 10

400000.0
Mean reward: 876.1288 Num episodes: 10


In [ ]:
#model.save("./200k_car_racing_PPO")
#model = PPO.load(f"./{numer}k_car_racing_PPO", env)

## Avalia o agente treinado

### Avaliação com visualização (jupyter notebook)

In [12]:
env = gym.make("CarRacing-v2", continuous=False, render_mode="human")
env = DummyVecEnv([lambda: env])
model = PPO.load(f"./{4e5}k_car_racing_PPO", env)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=3, render=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/home/melot/.local/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=909.17 +/- 28.898102211223577


In [13]:
env.close()

#### Resultado

In [14]:
df_treinos

,1,2,3,4,5,6,7,8,9,10
20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_treinos.T.astype(float).describe()

,20000.0,50000.0,100000.0,200000.0,400000.0
count,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN


### Prepare video recording

In [16]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [17]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

We will record a video using the [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper, you will learn about those wrapper in the next notebook.

In [23]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make("CarRacing-v2", render_mode="rgb_array", continuous=False)])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

### Visualize trained agent

In [69]:
# record_video("CarRacing-v2", model, video_length=500, prefix="ppo-car-racing")

NameError: name 'Monitor' is not defined

### Inicializar modelo salvo e grava video

In [25]:
for qtd in quantidades:
    model = PPO.load(f"{qtd}k_car_racing_PPO", env)
    record_video(f"{qtd}k_car_racing_PPO", model, video_length=600, prefix=f"ppo-{qtd}k_car_racing_PPO")

Saving video to /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-20000.0k_car_racing_PPO-step-0-to-step-600.mp4
Moviepy - Building video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-20000.0k_car_racing_PPO-step-0-to-step-600.mp4.
Moviepy - Writing video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-20000.0k_car_racing_PPO-step-0-to-step-600.mp4


Moviepy - Done !
Moviepy - video ready /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-20000.0k_car_racing_PPO-step-0-to-step-600.mp4
Saving video to /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-50000.0k_car_racing_PPO-step-0-to-step-600.mp4
Moviepy - Building video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-50000.0k_car_racing_PPO-step-0-to-step-600.mp4.
Moviepy - Writing video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-50000.0k_car_racing_PPO-step-0-to-step-600.mp4


Moviepy - Done !
Moviepy - video ready /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-50000.0k_car_racing_PPO-step-0-to-step-600.mp4
Saving video to /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-100000.0k_car_racing_PPO-step-0-to-step-600.mp4
Moviepy - Building video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-100000.0k_car_racing_PPO-step-0-to-step-600.mp4.
Moviepy - Writing video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-100000.0k_car_racing_PPO-step-0-to-step-600.mp4


Moviepy - Done !
Moviepy - video ready /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-100000.0k_car_racing_PPO-step-0-to-step-600.mp4
Saving video to /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-200000.0k_car_racing_PPO-step-0-to-step-600.mp4
Moviepy - Building video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-200000.0k_car_racing_PPO-step-0-to-step-600.mp4.
Moviepy - Writing video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-200000.0k_car_racing_PPO-step-0-to-step-600.mp4


Moviepy - Done !
Moviepy - video ready /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-200000.0k_car_racing_PPO-step-0-to-step-600.mp4
Saving video to /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-400000.0k_car_racing_PPO-step-0-to-step-600.mp4
Moviepy - Building video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-400000.0k_car_racing_PPO-step-0-to-step-600.mp4.
Moviepy - Writing video /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-400000.0k_car_racing_PPO-step-0-to-step-600.mp4


Moviepy - Done !
Moviepy - video ready /mnt/c/Users/matheus.melotti/Projects/ifes/repositories/inteligencia_artificial/Reinforcement_Learning/videos/ppo-400000.0k_car_racing_PPO-step-0-to-step-600.mp4


In [21]:
show_videos("videos", prefix="ppo")